In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os 
import psycopg2
from dotenv import load_dotenv,dotenv_values

In [3]:

base_url = 'https://quotes.toscrape.com'
pg_nm = 1
newlist = []
  # start from page 1

while pg_nm < 11:
    url = '/page/{0}/'.format(pg_nm)
    # print(base_url + url)
    new_url = base_url + url
    r = requests.get(new_url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        
        for quotes in soup.find_all('div',class_='quote'):
    
            quote = quotes.find('span', class_='text').text
            author = quotes.find('small', class_='author').text
            tags = quotes.find('div',class_='tags')

            alltags = []
            for ntag in tags.find_all('a'):
                alltags.append(ntag.text)

            data = {
                'Author':author,
                'Quote':quote,
                'tags':alltags
            }
            newlist.append(data)

    else:
        print(r)
    pg_nm +=1




In [4]:
df = pd.DataFrame(newlist)
# df.reset_index(drop=True)
df['id'] = range(1, len(df) + 1)

df = df[['id','Author','Quote','tags']]

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      100 non-null    int64 
 1   Author  100 non-null    object
 2   Quote   100 non-null    object
 3   tags    100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [5]:
df

,id,Author,Quote,tags
0,1,Albert Einstein,“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]"
1,2,J.K. Rowling,"“It is our choices, Harry, that show what we t...","[abilities, choices]"
2,3,Albert Einstein,“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]"
3,4,Jane Austen,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]"
4,5,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]"
...,...,...,...,...
95,96,Harper Lee,“You never really understand a person until yo...,[better-life-empathy]
96,97,Madeleine L'Engle,“You have to write the book that wants to be w...,"[books, children, difficult, grown-ups, write,..."
97,98,Mark Twain,“Never tell the truth to people who are not wo...,[truth]
98,99,Dr. Seuss,"“A person's a person, no matter how small.”",[inspirational]


In [6]:
author = df[['Author']].copy().drop_duplicates().reset_index(drop=True)
author['Authorid'] = range(1, len(author) + 1)
author = author[['Authorid','Author']]


author


,Authorid,Author
0,1,Albert Einstein
1,2,J.K. Rowling
2,3,Jane Austen
3,4,Marilyn Monroe
4,5,André Gide
5,6,Thomas A. Edison
6,7,Eleanor Roosevelt
7,8,Steve Martin
8,9,Bob Marley
9,10,Dr. Seuss


In [7]:
df = df.merge(author, on=['Author'],how='left')
# df['id'] = range(1, len(author) + 1)
df = df[['id','Authorid','Quote','tags']]

df

,id,Authorid,Quote,tags
0,1,1,“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]"
1,2,2,"“It is our choices, Harry, that show what we t...","[abilities, choices]"
2,3,1,“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]"
3,4,3,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]"
4,5,4,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]"
...,...,...,...,...
95,96,49,“You never really understand a person until yo...,[better-life-empathy]
96,97,50,“You have to write the book that wants to be w...,"[books, children, difficult, grown-ups, write,..."
97,98,14,“Never tell the truth to people who are not wo...,[truth]
98,99,10,"“A person's a person, no matter how small.”",[inspirational]


In [8]:
base_dir = './data'
df.to_csv(os.path.join(base_dir,'quotes.csv'),index=False)
author.to_csv(os.path.join(base_dir,'authors.csv'),index=False)

In [16]:
# Create database 

db_params = {
    'username':'postgres',
    'password': 'password',
    'host':'localhost',
    'port': '5432',
    'database':'toscrape'
}

default_db_url = f"postgresql://{db_params['username']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/postgres"


def createDb(dbname=db_params['database'],db_url=default_db_url):
    try:
        conn = psycopg2.connect(db_url)
        conn.autocommit = True 
        cur = conn.cursor()

        # check if the database exists

        cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{dbname}' ")
        exists = cur.fetchone()
        if not exists:
            # create the database
            cur.execute(f"CREATE DATABASE {dbname}")
            print(f"Database {dbname} successfully created")
        else:
            print(f"database {dbname} already exists")

        # cur.close()
    except Exception as e:
        print('error here ',e)

createDb()

database toscrape already exists


In [17]:
def getDbConection():
    connection = psycopg2.connect(
        host = db_params['host'],
        database = db_params['database'],
        user = db_params['username'],
        password = db_params['password']
    )

    return connection

    # connect to our database 

  

conn = getDbConection()

In [18]:
# create schema and tables 

def create_tables(sql_path,schema='toscrape'):
    conn = getDbConection()
    cursor = conn.cursor()


    # Read the SQL file
    with open(sql_path, 'r') as file:
        sql_commands = file.read()
    create_table_query = f" CREATE SCHEMA IF NOT EXISTS {schema}; \
                        {sql_commands} \
            "
    
    # print(create_table_query)
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
    # print(f"Tables created successfully {sql_path.split('/')[-1]}")

## create oRDER OF CREATING TABLES TO AVOID FOREIGN KEYS ERRORS
models = ['authors','quotes']
sqlfiles = os.listdir(os.path.join('sql'))
for model in models:
    for sql_file in sqlfiles:
        if model == sql_file.split('.')[0]:
            sql_file = os.path.join('sql', sql_file)

            create_tables(sql_file)
            print(f"Creating table from {sql_file}")

Creating table from sql/authors.sql
Creating table from sql/quotes.sql


In [12]:
# create tables in postgressql

### ASSUMED DB HAVE BEEN CREATED IN POSTGRESQL

In [19]:
def genPercentSym(data:tuple) -> str:

        d = []
        data = data.split(',')

        for i in range(len(data)):
            d.append('%s')


        return ', '.join(d)

In [21]:
import csv
import os

def load_data_from_csv(csv_path):
    conn = getDbConection()
    cursor = conn.cursor()
    columns = None
    query = None
    tablename = os.path.basename(csv_path).split('/')[-1].split('.')[0]
    schema = 'toscrape'
    
    with open(csv_path, 'r') as f:
        reader = csv.reader(f)
         # Skip header row
        # next(reader)
        for i,row in enumerate(reader):
            try:
                if i == 0:
                    row = [i.lower() for i in row]
                    columns = ', '.join(row)
                    continue
                else:
                    
                    query = f"INSERT INTO {schema}.{tablename} ({columns}) VALUES ({genPercentSym(columns)})"
                    cursor.execute(query, row)
            except psycopg2.IntegrityError: # foreign key violation
                conn.rollback() # rollback the current transaction or skip the transaction
            else:
                conn.commit() # commit if hte rows are inserted successfully

            
            

    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data loaded from {csv_path} successfully.")


## create oRDER OF CREATING TABLES TO AVOID FOREIGN KEYS ERRORS
models = ['authors','quotes']
csvfiles = os.listdir(os.path.join(base_dir))
# Load data into the database

for model in models:
    for csv_file in csvfiles:
        if model == csv_file.split('.')[0]:
            csv_file = os.path.join(base_dir, csv_file)
            load_data_from_csv(csv_file)
            print(csv_file,model)

Data loaded from ./data/authors.csv successfully.
./data/authors.csv authors
Data loaded from ./data/quotes.csv successfully.
./data/quotes.csv quotes
